In [91]:
import pandas as pd
import os
import numpy as np



In [92]:
## import raw data and show the first 5 rows

raw_data_file = '/home/jovyan/work/home/jovyan/echantillon.csv'

df = pd.read_csv(raw_data_file, sep = ';')

df.head()

,conv_id,tag_main,tag_detail,country,conversation
0,10927721,product,product_information,France,"visitor: bonjour, le sac a dos trail xp14 lady..."
1,10933035,product,product_information,France,visitor: bonjour quel magazin à paris à le plu...
2,10933115,product,product_help,France,visitor: comment faire pour rectifier une err...
3,10933279,product,product_help,France,"visitor: peut-on le mrttre e, mode veille?ex j..."
4,10933457,delivery,delivery,France,visitor: bj je voie pas mode livraison domicil...


In [93]:
## choose a scenario 

scenario = 'delivery'

scenario_df = df[df['tag_main'] == scenario]

print ("Types of tag_deteil: {}".format(set(delivery_df['tag_detail'].tolist())))


Types of tag_deteil: {'delivery_mode', 'delivery_international', 'delivery_problem', 'delivery', 'delivery_fees', 'delivery_time'}


In [165]:
## separate and extract Q&A pairs

scenario_df.head()


def extract_Q_A_pairs(lst):
    stripped_lst = [str(l).strip() for l in lst]
    Q_pointer = A_pointer = 0
    Q_list = []
    A_list = []
    
    while (Q_pointer < len(lst)):
    
        while('visitor:' not in stripped_lst[Q_pointer]):  ##lst contains no visitor message (i.e., no question)
                Q_pointer +=1
                if Q_pointer >= len(stripped_lst):
                    break

        if Q_pointer >= len(stripped_lst):              
            break
        else:
            Q_temp = stripped_lst[Q_pointer].split('visitor:')[1].strip()
            A_pointer = Q_pointer+1
            if A_pointer >= len(stripped_lst): ## lst contains only 1 visitor message but without operator message
                                               ## i.e., question without corresponding answer
                break
            else:
                while (A_pointer < len(lst)):
                    if 'operator:' not in stripped_lst[A_pointer]:
                        if 'visitor:' in stripped_lst[A_pointer]:
                            Q_temp = Q_temp + " " +  stripped_lst[A_pointer].split('visitor:')[1].strip()  
                        else:
                            Q_temp = Q_temp
                        A_pointer += 1
                    else:
                        break
                if A_pointer >= len(lst):
                    break
                else:
            
                    Q_list.append(Q_temp)
                    A_temp = stripped_lst[A_pointer].split('operator:')[1].strip() 
                    A_pointer = A_pointer +1
                    while (A_pointer < len(lst)):
                        if 'operator:' in stripped_lst[A_pointer]:
                            if 'operator:' in stripped_lst[A_pointer]:
                                A_temp = A_temp + " " + stripped_lst[A_pointer].split('operator:')[1].strip()
                            else:
                                A_temp = A_temp

                            A_pointer += 1
                        else:
                            break
                    A_list.append(A_temp)

                Q_pointer = A_pointer

                
    return  (pd.DataFrame(np.column_stack([Q_list, A_list]), columns = ['Question', 'Answer'])) #(Q_list, A_list)

    



In [166]:
import time

start = time.time()
file = '/home/jovyan/work/home/jovyan/Q_and_A_pairs.csv'

df_all = pd.concat([extract_Q_A_pairs(scenario_df.iloc[i, -1].split('\n')) for i in range(len(scenario_df))], ignore_index = True)

df_all.to_csv(file, sep = ',')

print ('time {}'.format(time.time()- start))



time 46.98971962928772
